# Data Cleaning and Preprocessing

In [16]:
!pip install TextBlob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [17]:
import pandas as pd
import numpy as np
import isodate
from isodate import parse_duration
from textblob import TextBlob

In [2]:
video_df = pd.read_csv("dataFolder/raw/redPillAnalytics.csv")

In [3]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,F5eSaabAAmk,Benjamin Seda,How to ACTUALLY Get a Girlfriend in 2025 (Full...,👉🏼 Get 1-3+ dates per week in 30 days (coachin...,"['how to flirt with a girl', 'dates', 'how to ...",2025-03-06T15:27:49Z,5034,254.0,NaN,27.0,PT15M4S,hd,False
1,xJ6b8CV-pQ0,Benjamin Seda,How to Find A 10/10 Girlfriend,👫 My 3 step formula to approach & attract wome...,"['how to flirt with a girl', 'dates', 'how to ...",2025-03-03T15:01:24Z,3346,330.0,NaN,22.0,PT59S,hd,False
2,kPhrei5S88U,Benjamin Seda,The Mistake 99% of Men Make That Keep Them Single,👫 My 3 step formula to approach & attract wome...,"['how to flirt with a girl', 'dates', 'how to ...",2025-03-01T14:45:07Z,2690,222.0,NaN,19.0,PT36S,hd,False
3,4ZnwTwLcAeM,Benjamin Seda,How to Always Get That 2nd Date,👫 My 3 step formula to approach & attract wome...,"['how to flirt with a girl', 'dates', 'how to ...",2025-02-27T14:15:00Z,4060,413.0,NaN,9.0,PT46S,hd,False
4,VW9-SBs6yIg,Benjamin Seda,The Donald Trump Method for Tinder (STEAL THIS),👫 My 3 step formula to approach & attract wome...,"['how to flirt with a girl', 'dates', 'how to ...",2025-02-26T13:45:03Z,6818,316.0,NaN,30.0,PT32S,hd,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13670,_X4P6L622-8,The Corbett Report (Unofficial),How Do I Defend Voluntarism? - Questions For C...,I have no affiliation with James Corbett or Th...,NaN,2022-05-25T13:22:06Z,393,15.0,NaN,2.0,PT28M13S,hd,False
13671,tkmZ4c2AOVY,The Corbett Report (Unofficial),The 5G Dragnet,I have no affiliation with James Corbett or Th...,NaN,2022-05-24T17:28:15Z,1226,61.0,NaN,2.0,PT25M44S,sd,False
13672,mr7itEUIVew,The Corbett Report (Unofficial),False Flags: The Secret History of Al Qaeda — ...,I have no affiliation with James Corbett or Th...,NaN,2022-05-24T17:01:10Z,5531,179.0,NaN,16.0,PT1H16M19S,sd,False
13673,ochRNyIDTE8,The Corbett Report (Unofficial),Episode 409 - False Flags: The Secret History ...,I have no affiliation with James Corbett or Th...,NaN,2022-05-24T17:01:06Z,6132,212.0,NaN,16.0,PT1H59M39S,sd,False


In [4]:
video_df.isnull().any()

video_id          False
channelTitle      False
title             False
description        True
tags               True
publishedAt       False
viewCount         False
likeCount          True
favouriteCount     True
commentCount       True
duration          False
definition        False
caption           False
dtype: bool

In [5]:
video_df.dtypes

video_id           object
channelTitle       object
title              object
description        object
tags               object
publishedAt        object
viewCount           int64
likeCount         float64
favouriteCount    float64
commentCount      float64
duration           object
definition         object
caption              bool
dtype: object

In [6]:
video_df.isnull().sum()
video_df.dropna()
video_df.isin([0, np.nan]).sum()

video_id              0
channelTitle          0
title                 0
description         394
tags               6314
publishedAt           0
viewCount             3
likeCount            23
favouriteCount    13675
commentCount        150
duration              0
definition            0
caption           13246
dtype: int64

In [7]:
video_df.isnull().sum()

video_id              0
channelTitle          0
title                 0
description         394
tags               6314
publishedAt           0
viewCount             0
likeCount            16
favouriteCount    13675
commentCount          9
duration              0
definition            0
caption               0
dtype: int64

In [8]:
video_df['description'] = video_df['description'].fillna("")
video_df['tags'] = video_df['tags'].apply(lambda x: x if isinstance(x, list) else [])
video_df['likeCount'] = video_df['likeCount'].fillna(0)
video_df['favouriteCount'] = video_df['favouriteCount'].fillna(0)
video_df['commentCount'] = video_df['commentCount'].fillna(0)

print(video_df.isnull().any())

video_id          False
channelTitle      False
title             False
description       False
tags              False
publishedAt       False
viewCount         False
likeCount         False
favouriteCount    False
commentCount      False
duration          False
definition        False
caption           False
dtype: bool


In [9]:
numeric_cols = ['viewCount', 'likeCount', 'favouriteCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

In [10]:
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])
video_df['publishDayName'] = video_df['publishedAt'].dt.strftime("%A")
video_df['publishedAt_timestamp'] = video_df['publishedAt'].astype(int) / 10**9
video_df['tagCount'] = video_df['tags'].apply(lambda x: len(x) if isinstance(x, list) else 0)

In [11]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,publishDayName,publishedAt_timestamp,tagCount
0,F5eSaabAAmk,Benjamin Seda,How to ACTUALLY Get a Girlfriend in 2025 (Full...,👉🏼 Get 1-3+ dates per week in 30 days (coachin...,[],2025-03-06 15:27:49+00:00,5034.0,254.0,0.0,27.0,PT15M4S,hd,False,Thursday,1.741275e+09,0
1,xJ6b8CV-pQ0,Benjamin Seda,How to Find A 10/10 Girlfriend,👫 My 3 step formula to approach & attract wome...,[],2025-03-03 15:01:24+00:00,3346.0,330.0,0.0,22.0,PT59S,hd,False,Monday,1.741014e+09,0
2,kPhrei5S88U,Benjamin Seda,The Mistake 99% of Men Make That Keep Them Single,👫 My 3 step formula to approach & attract wome...,[],2025-03-01 14:45:07+00:00,2690.0,222.0,0.0,19.0,PT36S,hd,False,Saturday,1.740840e+09,0
3,4ZnwTwLcAeM,Benjamin Seda,How to Always Get That 2nd Date,👫 My 3 step formula to approach & attract wome...,[],2025-02-27 14:15:00+00:00,4060.0,413.0,0.0,9.0,PT46S,hd,False,Thursday,1.740666e+09,0
4,VW9-SBs6yIg,Benjamin Seda,The Donald Trump Method for Tinder (STEAL THIS),👫 My 3 step formula to approach & attract wome...,[],2025-02-26 13:45:03+00:00,6818.0,316.0,0.0,30.0,PT32S,hd,False,Wednesday,1.740578e+09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13670,_X4P6L622-8,The Corbett Report (Unofficial),How Do I Defend Voluntarism? - Questions For C...,I have no affiliation with James Corbett or Th...,[],2022-05-25 13:22:06+00:00,393.0,15.0,0.0,2.0,PT28M13S,hd,False,Wednesday,1.653485e+09,0
13671,tkmZ4c2AOVY,The Corbett Report (Unofficial),The 5G Dragnet,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:28:15+00:00,1226.0,61.0,0.0,2.0,PT25M44S,sd,False,Tuesday,1.653413e+09,0
13672,mr7itEUIVew,The Corbett Report (Unofficial),False Flags: The Secret History of Al Qaeda — ...,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:01:10+00:00,5531.0,179.0,0.0,16.0,PT1H16M19S,sd,False,Tuesday,1.653412e+09,0
13673,ochRNyIDTE8,The Corbett Report (Unofficial),Episode 409 - False Flags: The Secret History ...,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:01:06+00:00,6132.0,212.0,0.0,16.0,PT1H59M39S,sd,False,Tuesday,1.653412e+09,0


In [12]:
channel_stats = video_df.groupby('channelTitle').agg({
    'viewCount': 'sum',
    'likeCount': 'sum',
    'commentCount': 'sum'
}).reset_index()

In [13]:
channel_stats['engagementRate'] = (channel_stats['likeCount'] + channel_stats['commentCount']) / channel_stats['viewCount']

In [18]:
video_df['duration_seconds'] = video_df['duration'].apply(lambda x: parse_duration(x).total_seconds())

In [19]:
video_df.groupby('channelTitle')['duration_seconds'].mean()

channelTitle
Alhpamales                           24.600000
Benjamin Seda                       195.985205
Better Bachelor                    1379.782349
Coach Corey Wayne                   659.063435
FreshandFit                        3706.578228
Jordan B Peterson                  2940.362724
Paul Joseph Watson | Перевод        113.166667
The Corbett Report (Unofficial)    2280.352601
The Distributist                   5392.291045
Name: duration_seconds, dtype: float64

In [20]:
video_df['view_per_like'] = video_df['viewCount'] / video_df['likeCount']
video_df['comment_duration_interaction'] = video_df['commentCount'] * video_df['duration_seconds']
video_df['view_per_like'] = video_df['view_per_like'].fillna(0)
video_df['popularity_score'] = video_df['viewCount'] + video_df['likeCount'] * 10 + video_df['commentCount'] * 20

In [21]:
video_df['title_sentiment'] = video_df['title'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [22]:
video_df['commentCount'].mean()

595.6180621572212

In [23]:
video_df[['viewCount', 'commentCount']].corr()

,viewCount,commentCount
viewCount,1.000000,0.618975
commentCount,0.618975,1.000000


In [24]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,...,definition,caption,publishDayName,publishedAt_timestamp,tagCount,duration_seconds,view_per_like,comment_duration_interaction,popularity_score,title_sentiment
0,F5eSaabAAmk,Benjamin Seda,How to ACTUALLY Get a Girlfriend in 2025 (Full...,👉🏼 Get 1-3+ dates per week in 30 days (coachin...,[],2025-03-06 15:27:49+00:00,5034.0,254.0,0.0,27.0,...,hd,False,Thursday,1.741275e+09,0,904.0,19.818898,24408.0,8114.0,0.175000
1,xJ6b8CV-pQ0,Benjamin Seda,How to Find A 10/10 Girlfriend,👫 My 3 step formula to approach & attract wome...,[],2025-03-03 15:01:24+00:00,3346.0,330.0,0.0,22.0,...,hd,False,Monday,1.741014e+09,0,59.0,10.139394,1298.0,7086.0,0.000000
2,kPhrei5S88U,Benjamin Seda,The Mistake 99% of Men Make That Keep Them Single,👫 My 3 step formula to approach & attract wome...,[],2025-03-01 14:45:07+00:00,2690.0,222.0,0.0,19.0,...,hd,False,Saturday,1.740840e+09,0,36.0,12.117117,684.0,5290.0,-0.071429
3,4ZnwTwLcAeM,Benjamin Seda,How to Always Get That 2nd Date,👫 My 3 step formula to approach & attract wome...,[],2025-02-27 14:15:00+00:00,4060.0,413.0,0.0,9.0,...,hd,False,Thursday,1.740666e+09,0,46.0,9.830508,414.0,8370.0,0.000000
4,VW9-SBs6yIg,Benjamin Seda,The Donald Trump Method for Tinder (STEAL THIS),👫 My 3 step formula to approach & attract wome...,[],2025-02-26 13:45:03+00:00,6818.0,316.0,0.0,30.0,...,hd,False,Wednesday,1.740578e+09,0,32.0,21.575949,960.0,10578.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13670,_X4P6L622-8,The Corbett Report (Unofficial),How Do I Defend Voluntarism? - Questions For C...,I have no affiliation with James Corbett or Th...,[],2022-05-25 13:22:06+00:00,393.0,15.0,0.0,2.0,...,hd,False,Wednesday,1.653485e+09,0,1693.0,26.200000,3386.0,583.0,0.000000
13671,tkmZ4c2AOVY,The Corbett Report (Unofficial),The 5G Dragnet,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:28:15+00:00,1226.0,61.0,0.0,2.0,...,sd,False,Tuesday,1.653413e+09,0,1544.0,20.098361,3088.0,1876.0,0.000000
13672,mr7itEUIVew,The Corbett Report (Unofficial),False Flags: The Secret History of Al Qaeda — ...,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:01:10+00:00,5531.0,179.0,0.0,16.0,...,sd,False,Tuesday,1.653412e+09,0,4579.0,30.899441,73264.0,7641.0,-0.400000
13673,ochRNyIDTE8,The Corbett Report (Unofficial),Episode 409 - False Flags: The Secret History ...,I have no affiliation with James Corbett or Th...,[],2022-05-24 17:01:06+00:00,6132.0,212.0,0.0,16.0,...,sd,False,Tuesday,1.653412e+09,0,7179.0,28.924528,114864.0,8572.0,-0.400000


In [25]:
channel_stats

,channelTitle,viewCount,likeCount,commentCount,engagementRate
0,Alhpamales,239.0,10.0,0.0,0.041841
1,Benjamin Seda,202773449.0,6456129.0,356756.0,0.033599
2,Better Bachelor,231797826.0,12936057.0,2997777.0,0.068740
3,Coach Corey Wayne,251196988.0,4876107.0,396843.0,0.020991
4,FreshandFit,221150653.0,9730118.0,819214.0,0.047702
5,Jordan B Peterson,950219566.0,31148819.0,3469047.0,0.036431
6,Paul Joseph Watson | Перевод,2178.0,44.0,0.0,0.020202
7,The Corbett Report (Unofficial),1124728.0,84132.0,12672.0,0.086069
8,The Distributist,6873745.0,248340.0,92768.0,0.049625


In [30]:
print(top_10_views)

                                                   title   viewCount  \
9910           JBP X @MattRifeComedy.  Today at 5pm EST.  18502980.0   
10451  Lecture: Biblical Series I: Introduction to th...  13706650.0   
1463   THIS is How A Girl Wants You to TEXT HER | How...  11807649.0   
1473   7 Ways To INSTANTLY Look MORE ATTRACTIVE | How...   8246244.0   
9984                             COVID-19 Cause of Death   7952713.0   
10053  Talking to Muslims About Christ | Mohammed Hij...   7756906.0   
9973                            What Are Women Good For?   7656196.0   
10072         Africa is Not Poor Because of Colonization   7541337.0   
10389  Documentary: A Glitch in the Matrix (David Ful...   6605578.0   
90             If a Girl is Looking at You, Approach Her   6594217.0   

            channelTitle  
9910   Jordan B Peterson  
10451  Jordan B Peterson  
1463       Benjamin Seda  
1473       Benjamin Seda  
9984   Jordan B Peterson  
10053  Jordan B Peterson  
9973   Jordan B Pe

In [31]:
print(top_10_likes)

                                                   title  likeCount  \
9910           JBP X @MattRifeComedy.  Today at 5pm EST.   611913.0   
10290                                        Return Home   543720.0   
9955   The Fight Against Worldwide Child Slavery & th...   318009.0   
10110                               Article: Twitter Ban   315518.0   
9973                            What Are Women Good For?   305820.0   
9984                             COVID-19 Cause of Death   299559.0   
10087    Language Is Used as a Group Protection Strategy   232440.0   
114                 If A Girl is Looking at You, Do This   231402.0   
10451  Lecture: Biblical Series I: Introduction to th...   227120.0   
10072         Africa is Not Poor Because of Colonization   220386.0   

            channelTitle  
9910   Jordan B Peterson  
10290  Jordan B Peterson  
9955   Jordan B Peterson  
10110  Jordan B Peterson  
9973   Jordan B Peterson  
9984   Jordan B Peterson  
10087  Jordan B Peterson  
11